<a href="https://colab.research.google.com/github/kyleborot/lstm-and-sentiment-analysis/blob/main/LLaMa3_1_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install transformers datasets accelerate evaluate bitsandbytes huggingface_hub trl peft torch tensorboard

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

In [2]:
import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import evaluate
from IPython.core.display import HTML
import requests
import time
import datetime
import nltk
import re
HTML("""<style>.container{width:100% !important;} </style>""")

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score, precision_recall_fscore_support

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from datasets import Dataset, DatasetDict
from torch.utils.data import Dataset as tds, DataLoader as tdl
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import RobertaTokenizer, RobertaForSequenceClassification, DataCollatorWithPadding, RobertaModel, RobertaForMaskedLM, DataCollatorForLanguageModeling
import torch.nn as nn

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
with open('/content/drive/MyDrive/News Classification Kaggle/News_Category_Dataset_v3.json') as f:
  df = pd.read_json(f, orient='records', lines=True)

In [4]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(text):
  encodings = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
  return {
      'input_ids': encodings['input_ids'].squeeze().tolist(),
      'attention_mask': encodings['attention_mask'].squeeze().tolist()
  }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
df['text'] = df['headline'] + ' ' + df['short_description']
tokens = df['text'].apply(lambda x: tokenize_function(x))
tokens_df = pd.DataFrame(tokens.tolist())
tokens_df.columns = ['tokens_input_ids', 'tokens_attention_mask']
df = df.join(tokens_df)
df = df.drop(columns=['text'])
df = df.rename(columns={'tokens_input_ids': 'input_ids', 'tokens_attention_mask': 'attention_mask'})


In [6]:
df = df.loc[:,~df.columns.duplicated()]

In [7]:
def to_list(data):
  if isinstance(data, torch.Tensor):
    return data.tolist()
  return data
df['input_ids'] = df['input_ids'].apply(to_list)
df['attention_mask'] = df['attention_mask'].apply(to_list)

In [8]:
def convert_to_dataset(df):
  return Dataset.from_pandas(df)

In [9]:
required_columns = ['input_ids', 'attention_mask']
for col in required_columns:
  if col not in df.columns:
    raise ValueError(f"Required column '{col}' not found in the DataFrame.")

In [10]:
dataset = convert_to_dataset(df)

In [16]:
train_df, eval_df = train_test_split(df, test_size=0.1, random_state=42)

In [17]:
train_dataset = convert_to_dataset(train_df)
eval_dataset = convert_to_dataset(eval_df)

In [24]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    evaluation_strategy='epoch',
    save_total_limit=2,
    learning_rate=2e-5,
    gradient_accumulation_steps=4,
    fp16=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
model = RobertaForMaskedLM.from_pretrained("roberta-base")

In [26]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'precision': precision_recall_fscore_support(p.label_ids, preds, average='weighted')[0],
        'recall': precision_recall_fscore_support(p.label_ids, preds, average='weighted')[1],
        'f1': precision_recall_fscore_support(p.label_ids, preds, average='weighted')[2],
    }

In [27]:
from transformers import DataCollatorForLanguageModeling
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer)
)

In [ ]:
torch.cuda.empty_cache()
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained('/content/drive/MyDrive/News Classification Kaggle/roberta_model')
tokenizer.save_pretrained('/content/drive/MyDrive/News Classification Kaggle/roberta_tokenizer')

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)